In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import pandas_market_calendars as mcal
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Filtro de datas
start_date = pd.Timestamp(date(date.today().year, 5, 30))
end_date = pd.Timestamp(date.today())

# Calendário de feriados da b3
B3 = mcal.get_calendar('B3')
B3_holidays = B3.holidays()

# Filtro correto por range
feriados = [h for h in B3_holidays.holidays if start_date <= h <= end_date]

# Define um CustomBusinessDay com esses feriados
bday_brasil = pd.offsets.CustomBusinessDay(holidays=feriados)

# data d-2
d2_date = (date.today()-2*bday_brasil).strftime('%Y-%m-%d')
# d2_date = '2025-06-02'

date_range = pd.date_range(start=start_date, end=d2_date, freq='B')
date_range_no_holidays = date_range[~date_range.isin(feriados)]

Configuração de API e funções que coletam os dados

In [3]:
# User configs for API authentication
api_acces_id = "89297662e92386720e192e56ffdc0d5e.access"
api_secret = "b8b3cfabf25982a64a1074360f83b0dc143aa5bd75560abf5c901b0977364de4"
api_password = "juNTr1QbtbY9NZ8ACrMF"
user_name = "alberto.coppola@perenneinvestimentos.com.br"

columns_filter = ['portfolio_id','overview_date', 'instrument_name', 'instrument_id','book_name', 'instrument_type', 'quantity', 'price', 'asset_value','exposure_value','dtd_ativo_fin']


# Retorna um dataframe a partir da parametros
def fetch_data(url: str, params: dict, access_id, secret, user_name, api_password):
    # trocar para a URL do ambiente desejado
    base_url = "https://perenne.bluedeck.com.br/api"
    url_token = "auth/token"
    
    # trocar o e-mail e a senha de aplicação
    data = {"username": user_name, "password": api_password}

    # trocar o ID e o secret
    client_headers = {
        "CF-Access-Client-Id": access_id,
        "CF-Access-Client-Secret": secret
    }

    # realiza request
    response = requests.post(f"{base_url}/{url_token}", data=data, headers=client_headers)

    # Converte em json
    json_response = response.json()

    # token de acesso
    access_token = json_response['access_token']
    token_type = json_response['token_type']
    # expiration_dt = json_response['expires_at']

    # Definindo headers para realização de chamadas
    client_header = {
        "CF-Access-Client-Id": access_id,
        "CF-Access-Client-Secret": secret
    }
    token_header = {"Authorization": f"{token_type} {access_token}"}
    headers = {**client_header, **token_header}

    response_api = requests.post(f"{base_url}/{url}", headers=headers, json=params)
    response_json = response_api.json()
    
    return response_json

Busco todas posições de ativos

In [4]:
# URL de posições
url = 'portfolio_position/positions/get'

# define parametros de busca
params = {
    "start_date": "2025-06-30",
    "end_date": d2_date,
    "portfolio_group_ids": [1],
}

df_positions = fetch_data(url, params, api_acces_id, api_secret, user_name, api_password)['objects']
df_positions = pd.DataFrame(df_positions)
# Salva o de-para de portfolio_id e nome
funds_nav = df_positions.loc[['portfolio_id','date','net_asset_value']].T.drop_duplicates()
funds_nav.rename(columns={"net_asset_value":"portfolio_nav"},inplace=True)

Busco todos os fundos que são explodidos e geridos internamente

In [5]:
# define URL de acesso
url_funds = 'portfolio_registration/portfolio_group/get'

# define parametros de busca
params_funds = {"get_composition": True}

# Identifica todos os fundos do sistema para explodi-los
all_funds = fetch_data(url_funds, params_funds, api_acces_id, api_secret, user_name, api_password)['objects']
all_funds = pd.DataFrame(all_funds)

all_funds = list(all_funds['8'].loc['composition_names'].values())

funds_name = df_positions.loc[['portfolio_id','name']].T.drop_duplicates()
funds_name.to_csv("funds_name.csv", index=False)

Histórico dos benchmarks (CDI, IBOV e IHFA)

In [6]:
# define URL de acesso
url = 'market_data/pricing/prices/get'

# define parametros de busca
params = {
  "start_date": "2024-12-31",
  "end_date": d2_date,
  "instrument_ids": [
    1540,
    1932,
    9
  ]
}

# Identifica todos os fundos do sistema para explodi-los
bench_df = fetch_data(url, params, api_acces_id, api_secret, user_name, api_password)
bench_df = pd.DataFrame(bench_df['prices'])[['date','instrument','variation']]
bench_df.variation = bench_df.variation.astype(np.float64)
bench_df['date'] = pd.to_datetime(bench_df['date'])
bench_df = bench_df.sort_values(['date'])

# YTD percentual
bench_df['ytd_pct'] = bench_df.groupby(
    ['instrument'])['variation'].transform(lambda x: (1 + x).cumprod() - 1)
bench_df.to_csv("benchmarks.csv",index=False)

Busco por erros de PNL pelo sistema

In [7]:
url = 'portfolio_position/attribution/errors_view/get'

for f,g in funds_name.iterrows():
    if g['name'] in all_funds:

        params = {
        "base_date": d2_date,
        "consolidation_type": 3,
        "show_errors": True,
        "periods": [
            2,
            3
        ],
        "attribution_types": [
            1
        ],
        "portfolio_ids": [
            g.portfolio_id
        ]
        }
        errors = fetch_data(url, params, api_acces_id, api_secret, user_name, api_password)['objects']
        errors = pd.DataFrame(errors).T

        if len(errors)>=1:
            errors = (errors).merge(funds_name,on='portfolio_id',how='left')[['name','date']]
            print("Erro encontrado:")
            print(errors)

Erro encontrado:
             name        date
0  CSHG YANKEE II  2025-07-08


Construção do CSV de posições e custos

In [8]:
registros = pd.DataFrame()
costs_df = pd.DataFrame()

# Iterar pelas colunas de posição (ids)
for data_col in df_positions.columns:

    try:
        # Pega os dados do id (coluna), vira Series
        id_col = df_positions[data_col]

        # Pega o portfolio_id
        portfolio_id = id_col['portfolio_id']

        # Pula o portfolio consolidado para evitar duplicidade 
        if portfolio_id == 49:
            continue

        # Pega o dicionário de instrument_positions e transforma em df
        instrument_positions = pd.DataFrame(id_col['instrument_positions'])

        # Pega o dicionário de custos e transforma em df
        costs_position = pd.DataFrame(id_col['financial_transaction_positions'])

        # Concatena o portfolio id
        instrument_positions['portfolio_id'] = portfolio_id

        # Busca a lista de pnl dentro da coluna Attribution e concatena pro df de instrument_position
        pnl_ = []
        for _, row in instrument_positions.iterrows():
            try:
                pnl_.append(np.float64(row.attribution['total']['financial_value']))
            except:
                pnl_.append(0.0)

        instrument_positions['dtd_ativo_fin'] = pnl_

        # Faz o mesmo pra custos
        pnl_ = []
        for _, row in costs_position.iterrows():
            try:
                pnl_.append(np.float64(row.attribution['total']['financial_value']))
            except:
                pnl_.append(0.0)

        costs_position['dtd_custos_fin'] = pnl_

        # Concatena a data nos custos
        costs_position['overview_date'] = instrument_positions.overview_date.unique()[0]
        costs_position['origin_portfolio_id'] = portfolio_id

        # Concatena no df de registros
        registros = pd.concat([registros,instrument_positions])

        # Concatena no df de custos
        costs_df = pd.concat([costs_df,costs_position])
    
    except Exception as e:
        print(f"Erro ao processar data {data_col}: {e}")
    

# Filtro pelas colunas de interesse e transforma quantidade, preço e aum em float
registros = registros[columns_filter]
registros.loc[:,['quantity','asset_value','exposure_value','price','dtd_ativo_fin']] = registros[['quantity','asset_value','exposure_value','price','dtd_ativo_fin']].astype(np.float64)

# Transforma a coluna de custos em float e o id em int
costs_df['financial_value']=costs_df['financial_value'].astype(float)

# Filtro as colunas de custos
costs_df = costs_df[['financial_value','attribution','book_name','category_name','origin_portfolio_id','origin_accounting_transaction_id','dtd_custos_fin','overview_date']]

Uso interno para pegar books e registros duplicados

In [9]:
#Exceções
registros['book_name'] = registros['book_name'].replace("Risco >> HYPE >> Ação HYPE", "Risco >> HYPE")
registros['book_name'] = registros['book_name'].replace("Risco >> HYPE >> Opção HYPE", "Risco >> HYPE")

In [10]:
#informaçoes de cadastro de book dos fundos que explodimos
dup = registros.merge(funds_name,on='portfolio_id',how='left')[['name','instrument_name','book_name']].drop_duplicates()
dup = dup[dup['name'].isin(all_funds)]

# Conta quantos books diferentes cada (portfolio_id, instrument_name) tem
duplicados = (
    dup
    .groupby(['name', 'instrument_name'])['book_name']
    .nunique()
    .reset_index()
)

# Seleciona só os que aparecem em 2 ou mais books
duplicados = duplicados[duplicados['book_name'] > 1][['name', 'instrument_name']]
duplicados = duplicados.merge(dup,on=['name','instrument_name'],how='left')
display(duplicados)

"""
----------- Filtro de contingencia enquanto não arruma as categorias no sistema ---------

        soma as quantidades e notional pegando o primeiro book name para categorização
"""
registros = registros.groupby(['overview_date','portfolio_id','instrument_name']).agg({
    'instrument_id':'first',
    'book_name':'first',
    'instrument_type':'first',
    'quantity':'sum',
    'price':'first',
    'asset_value':'sum',
    'exposure_value':'sum',
    'dtd_ativo_fin': 'sum'
}).reset_index()

,name,instrument_name,book_name


Flatten o csv de posições em um novo csv 'explodido'

In [11]:
# Adiciono uma coluna com o NAV do fundo para o calculo de PNL
registros = pd.merge(registros,funds_nav,left_on=['portfolio_id','overview_date'],right_on=['portfolio_id','date']).drop(columns='date')
registros.rename(columns={"net_asset_value":"portfolio_nav"},inplace=True)

In [12]:
def explodir_portfolio(portfolio_id, data, todas_posicoes, todos_custos, visitados=None, notional = None, portfolio_origem_id=None, nivel = 0):
    """
    - portfolio_id: o portfólio que estamos processando
    - data: a data da posição
    - todas_posicoes: DataFrame com todas as posições
    - multiplicador: proporção da posição herdada
    """
    
    if visitados is None:
        visitados = set()
        # print("STARTING: ",portfolio_id)
    if portfolio_id in visitados:
        return [], pd.DataFrame()

    visitados.add(portfolio_id)

    # Filtra as posições desse portfolio na data
    posicoes = todas_posicoes[
        (todas_posicoes['overview_date'] == data) &
        (todas_posicoes['portfolio_id'] == portfolio_id)
    ]

    # Filtra os custos do portfolio na data
    custos = todos_custos[
        (todos_custos['overview_date'] == data) &
        (todos_custos['origin_portfolio_id'] == portfolio_id)
    ]
    

    # Calculo do AUM total do fundo, soma ativos + custos
    aum = posicoes.asset_value.sum() + custos.financial_value.sum()
    # print(aum)
    if notional is None:
        notional = aum
    # print(notional)
    # Calculo um multiplicador para proporcionalizar as posições
    mult = notional/aum

    if portfolio_origem_id is None:
        portfolio_origem_id = portfolio_id
        # exposure = 1
    else:
        # print("Multiplicando por: ", round(exposure*100,4), "%")
        posicoes.loc[:,['quantity','asset_value','exposure_value','dtd_ativo_fin']] = posicoes[['quantity','asset_value','exposure_value','dtd_ativo_fin']] * mult
        custos.loc[:,['financial_value','dtd_custos_fin']] = custos.loc[:,['financial_value','dtd_custos_fin']] * mult

    # Seta o portfolio_id para a origem e cria na tabela de custos
    posicoes.loc[:,['portfolio_id']] = portfolio_origem_id
    custos.loc[:,['root_portfolio']] = portfolio_origem_id

    resultados = []

    for _, row in posicoes.iterrows():
        row_portfolio_id = row['instrument_id']
        if row.instrument_name in (all_funds):  # Checa se a linha é um fundo
            # print(row.instrument_name)
            # É um fundo investido, explodir recursivamente
            sub_resultados, resultado_custo = explodir_portfolio(
                row_portfolio_id,
                data,
                todas_posicoes,
                todos_custos,
                visitados=visitados,
                notional = np.float64(row.asset_value),
                portfolio_origem_id=portfolio_origem_id,
                nivel = nivel + 1
            )
            resultados += sub_resultados
            # Concatena no df de custos
            custos = pd.concat([custos,resultado_custo])
        else:
            novo = row.copy()
            novo['portfolio_origem'] = portfolio_id
            novo["nivel"] = nivel
            resultados.append(novo)
    
    return resultados, custos

todas_explodidas = pd.DataFrame()
todos_custos_explodidos = pd.DataFrame()

datas = registros.overview_date.unique()
portfolios = registros['portfolio_id'].unique()

for data in datas:
    for portfolio in portfolios:
        explodido, custo = explodir_portfolio(portfolio, data, registros, costs_df)
        todas_explodidas = pd.concat([todas_explodidas,pd.DataFrame(explodido)])
        todos_custos_explodidos = pd.concat([todos_custos_explodidos,custo])
df_explodido = pd.DataFrame(todas_explodidas)

# Subscrevo o portfolio_nav para referenciar ao portfolio_id original
df_explodido = pd.merge(df_explodido.drop(columns=['portfolio_nav']),funds_nav,left_on=['portfolio_id','overview_date'],right_on=['portfolio_id','date'],how='left').drop(columns=['date'])

todos_custos_explodidos = todos_custos_explodidos.groupby(['overview_date','root_portfolio','origin_portfolio_id','category_name']).agg({
    'book_name':'first',
    'financial_value':'sum',
    'dtd_custos_fin':'sum'
}).reset_index()
todos_custos_explodidos['overview_date'] = pd.to_datetime(todos_custos_explodidos['overview_date'])

# df_explodido['pct_exposicao'] = df_explodido['exposure_value'].astype(float) / df_explodido['portfolio_nav'].astype(float)

# df_explodido.drop(columns='instrument_name',inplace=True)
df_explodido['overview_date'] = pd.to_datetime(df_explodido['overview_date'])

df_explodido = df_explodido.groupby(['overview_date','portfolio_id','instrument_name','instrument_id']).agg({
        'book_name':'first',
        'asset_value':'sum',
        'dtd_ativo_fin':'sum',
        'exposure_value':'sum'
    }).reset_index()

C:\Users\AlbertoCoppola\AppData\Local\Temp\ipykernel_17908\3120038746.py:37: RuntimeWarning: invalid value encountered in scalar divide
  mult = notional/aum
C:\Users\AlbertoCoppola\AppData\Local\Temp\ipykernel_17908\3120038746.py:37: RuntimeWarning: invalid value encountered in scalar divide
  mult = notional/aum
C:\Users\AlbertoCoppola\AppData\Local\Temp\ipykernel_17908\3120038746.py:37: RuntimeWarning: invalid value encountered in scalar divide
  mult = notional/aum


Tratamento de exceção

In [13]:
# posiçoes fora do relatorio e custos viram mesmo book e sao concatenados a partir de 30 de junho
todos_custos_explodidos.loc[:,'book_name'] = 'Risco >> Caixas e Provisionamentos >> CPR (Provisões)'
todos_custos_explodidos.rename(columns={'root_portfolio':'portfolio_id','category_name':'instrument_name','dtd_custos_fin':'dtd_ativo_fin','financial_value':'asset_value'},inplace=True)

In [63]:
try:
    print("Reading feed file...")
    britechdf = pd.read_csv('feed_britech.csv',encoding='latin',parse_dates=True).dropna(how='all')
    britechdf[['asset_value','dtd_ativo_pct','dtd_ativo_fin','exposure_value']]=britechdf[['asset_value','dtd_ativo_pct','dtd_ativo_fin','exposure_value']].astype(float)
    britechdf['overview_date'] = pd.to_datetime(britechdf['overview_date'])

    betacurve = pd.read_csv('beta_curva.csv',encoding='latin',parse_dates=True).dropna(how='all')
    betacurve[['asset_value','dtd_ativo_pct','dtd_ativo_fin','exposure_value']]=betacurve[['asset_value','dtd_ativo_pct','dtd_ativo_fin','exposure_value']].astype(float)
    betacurve['overview_date'] = pd.to_datetime(betacurve['overview_date'])
except Exception as e:
        input(f"{e}")

Reading feed file...


In [64]:
df_final = pd.DataFrame()
for ptf in britechdf.portfolio_id.unique():
    """
    - filtra o dataframe de posições de acordo com o csv de feed (britechdf).
    - constrói um novo dataframe reorganizando e concatenando os books de posições e custos
    - esse dataframe é o de caixa e provisionamentos da carteira
    """
    filtered_britechdf = britechdf[britechdf.portfolio_id==ptf]
    filtered_beta = betacurve[betacurve.portfolio_id==ptf]

    df_cpr = df_explodido[~(df_explodido['book_name'].isin(list(filtered_britechdf.book_name.drop_duplicates()))) & 
    (df_explodido['portfolio_id']== ptf) &
    ~(df_explodido['book_name'].str.lower().str.startswith('off'))]

    df_cpr.loc[:,'book_name'] = 'Risco >> Caixas e Provisionamentos >> CPR (Provisões)'

    df_cpr = pd.concat([df_cpr,todos_custos_explodidos[todos_custos_explodidos.portfolio_id==ptf]],ignore_index=True)
    df_cpr = df_cpr.loc[df_cpr['overview_date'] > pd.Timestamp('2025-06-30')]
    
    df_cpr['asset_value_ontem'] = (
    df_cpr.groupby(['portfolio_id','instrument_name'])['asset_value']
    .shift(1)
)

    df_cpr = df_cpr.groupby(['overview_date','portfolio_id','instrument_name']).agg({
        'book_name':'first',
        'asset_value':'sum',
        'asset_value_ontem':'sum',
        'dtd_ativo_fin':'sum'
    }).reset_index()

    cpr_britech = filtered_britechdf[(filtered_britechdf['book_name']=='Risco >> Caixas e Provisionamentos')]
    df_cpr = pd.concat([df_cpr,cpr_britech],ignore_index=True).reset_index(drop=True)

    df_positions = df_explodido[(df_explodido['book_name'].isin(list(filtered_britechdf.book_name.drop_duplicates()))) & 
                    (df_explodido['portfolio_id']== ptf)]
     
    df_positions['exposure_value_ontem'] = (
        df_positions.groupby(['portfolio_id','instrument_name'])['exposure_value']
        .shift(1)
    )

    df_positions['asset_value_ontem'] = (
        df_positions.groupby(['portfolio_id','instrument_name'])['asset_value']
        .shift(1)
    )
    df_positions = df_positions.dropna()

    pos_britech = filtered_britechdf[~(filtered_britechdf['book_name']=='Risco >> Caixas e Provisionamentos')]
    df_positions = pd.concat([df_positions,pos_britech],ignore_index=True).reset_index(drop=True)
    df_positions = pd.concat([df_positions,filtered_beta],ignore_index=True).reset_index(drop=True)

    df_positions = df_positions.sort_values(['portfolio_id','instrument_name','overview_date'])

    # Calculo do pnl diário do ativo e da carteira
    df_positions['dtd_ativo_pct'] = df_positions['dtd_ativo_fin'] / df_positions['exposure_value_ontem']

    df_positions['Year'] = df_positions['overview_date'].dt.year
    df_positions['Month'] = df_positions['overview_date'].dt.month

    # MTD percentual
    df_positions['mtd_ativo_pct'] = df_positions.groupby(
        ['portfolio_id','instrument_name','Year','Month'])['dtd_ativo_pct'].transform(lambda x: (1 + x).cumprod() - 1)

    # YTD percentual
    df_positions['ytd_ativo_pct'] = df_positions.groupby(
        ['portfolio_id','instrument_name','Year'])['dtd_ativo_pct'].transform(lambda x: (1 + x).cumprod() - 1)

    df_positions = pd.concat([df_positions,df_cpr],ignore_index=True)
    df_positions['overview_date'] = pd.to_datetime(df_positions['overview_date'])
    df_positions = df_positions.fillna(0)

    df_final = pd.concat([df_final,df_positions])
df_final = df_final.sort_values(['portfolio_id','instrument_name','overview_date'])


C:\Users\AlbertoCoppola\AppData\Local\Temp\ipykernel_17908\3977027211.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positions['exposure_value_ontem'] = (
C:\Users\AlbertoCoppola\AppData\Local\Temp\ipykernel_17908\3977027211.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positions['asset_value_ontem'] = (


Quebra os grupos em colunas

In [65]:
# Split da coluna pelo separador " >> "
df_split = df_final['book_name'].str.split(' >> ', expand=True)

# Renomeia as colunas de acordo com a quantidade de splits encontrados
df_split.columns = [f'grupo_{i+1}' for i in range(df_split.shape[1])]

groups_df = pd.concat([df_split,df_final['book_name']],axis=1).drop_duplicates()
groups_df = pd.concat([groups_df,pd.DataFrame({'grupo_1':['Caixa','Risco'],'book_name':['Caixa','Risco']})])

df_all = pd.concat([df_final, df_split], axis=1)
df_all = df_all.replace(np.inf,0).replace(-np.inf,0)

groups_df.fillna("").to_csv("groups.csv", index=False)
df_all.to_csv('positions.csv', index=False)